<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
💰 תהליך הסבת שכבת גושי שומה לבנק"ל מודרני 💰 

In [1]:
print('2. Tax Assessment Blocks Borders:')

2. Tax Assessment Blocks Borders:


<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess
from configs import CNFG

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [3]:
print('  2.a Reading inputs')

  2.a Reading inputs


<h1 style="text-align:right;font-size:100%">
רשימת גושי שומה

In [4]:
Tax_List = pd.read_csv(CNFG.Local + 'GushShumaList.csv')

<h1 style="text-align:right;font-size:100%">
טבלת גושי שומה

In [5]:
table_cols = ['GUSH_SHUMA_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'BOOK_COUNTY', 'BOOK_COMMENT', 'LOCALITY_ID']

Tax_Table = pd.read_csv(CNFG.Oracle +'gush_shuma.csv', usecols = table_cols, encoding = 'cp1255')

<h1 style="text-align:right;font-size:100%">
שכבת גושי שומה

In [6]:
layer_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

Tax_Layer = gpd.read_file(CNFG.CADSDE + 'SUB_GUSH_ALL_SHUMA.gdb',
                          layer = 'SUB_GUSH_ALL_SHUMA',
                          include_fields = layer_cols)

Tax_Layer.rename(columns={'GUSH_NUM':'BlockNumber', 'GUSH_SUFFIX':'SubBlockNumber', 'STATUS':'BlockStatus'}, inplace=True)

In [7]:
del[table_cols, layer_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️
<h1 style="text-align:right; font-size:125%">
<h1 style="text-align:right; font-size:100%">
מילוי ערכי תת-גוש בספרה 0 עבור מקורות המידע
<h1 style="text-align:right; font-size:100%">    
הוספת שדה מקשר לפי מספר גוש מלא בכל מקורות המידע

In [8]:
print('  2.b Cleaning data')

  2.b Cleaning data


In [9]:
Tax_Table['GUSH_SUFFIX'] = Tax_Table['GUSH_SUFFIX'].fillna(0).astype(int)
Tax_Layer['SubBlockNumber'] = Tax_Layer['SubBlockNumber'].fillna(0).astype(int)


Tax_Table['BlockName'] = Tax_Table['GUSH_NUM'].astype(str) + '_' + Tax_Table['GUSH_SUFFIX'].astype(str)
Tax_Layer['BlockName'] = Tax_Layer['BlockNumber'].astype(str) + '_' + Tax_Layer['SubBlockNumber'].astype(str)
Tax_List['BlockName'] = Tax_List['gush_id'] + '0'

<h1 style="text-align:right; font-size:100%">
מיזוג נתוני רשימת שומה עם טבלת שומה
שינוי שמות של שדות

In [10]:
Merged_Table = Tax_List.merge(Tax_Table, on = 'BlockName', how = 'left')
Merged_Table.rename(columns = {'GUSH_SHUMA_ID' : 'BlockUniqueID'}, inplace = True)

<h1 style="text-align:right; font-size:100%">
מילוי ערכי מספר מזהה של גוש שומה לפי המשכיות של מספר רץ

In [11]:
biggest = Merged_Table['BlockUniqueID'].max() + 1

Merged_Table['BlockUniqueID'].loc[Merged_Table['BlockUniqueID'].isnull()] = biggest
Merged_Table['rank'] = Merged_Table.groupby('BlockUniqueID')['BlockUniqueID'].rank('first')
Merged_Table['rank'] = Merged_Table['rank'] - 1
Merged_Table['BlockUniqueID'].loc[Merged_Table['BlockUniqueID'] == biggest] = Merged_Table['rank'] + Merged_Table['BlockUniqueID']
Merged_Table = Merged_Table.drop(['rank'],axis = 1)

In [12]:
print('  2.c Data conversion')

  2.c Data conversion


<h1 style="text-align:right; font-size:100%">
מיזוג תוצאה קודמת עם שכבת גושי שומה

In [13]:
Merged_layer = Merged_Table.merge(right = Tax_Layer,
                                  on  = 'BlockName',
                                  how = 'outer',
                                  indicator = True,
                                  suffixes = ('','_layer'))

<h1 style="text-align:right; font-size:100%">
תיקון והשלמה של שדה מספר גוש ומספר תת גוש 
<h1 style="text-align:right; font-size:100%">
הסרת שדות שאינם רלוונטיים
<h1 style="text-align:right; font-size:100%">
מיון רשומות לפי מספר זיהוי והשלמת מספרים מזהים חסרים לפי רציפות מספרית
<h1 style="text-align:right; font-size:100%">
הסרת גושים כפולים

In [14]:
Merged_layer['SubBlockNumber'] = Merged_layer['SubBlockNumber'].fillna(Merged_layer['GUSH_SUFFIX'])
Merged_layer['SubBlockNumber'] = Merged_layer['SubBlockNumber'].fillna(0)
Merged_layer['SubBlockNumber'] = Merged_layer['SubBlockNumber'].astype('Int64')

Merged_layer['BlockNumber'] = Merged_layer['BlockNumber'].fillna(Merged_layer['GUSH_NUM'])
Merged_layer['BlockNumber'] = Merged_layer['BlockNumber'].fillna(Merged_layer['gush_shuma_num'])
Merged_layer['BlockNumber'] = Merged_layer['BlockNumber'].astype('Int64')

Merged_layer.drop(columns = ['gush_shuma_num','GUSH_SUFFIX', 'GUSH_NUM', '_merge', 'gush_id'], inplace=True)

In [15]:
Merged_layer['BlockUniqueID'] = Merged_layer['BlockUniqueID'].astype('Int64')
Merged_layer.sort_values('BlockUniqueID', inplace=True)
Merged_layer['BlockUniqueID'] = Merged_layer['BlockUniqueID'].ffill()+Merged_layer.groupby(Merged_layer.BlockUniqueID.notnull().cumsum()).cumcount()

In [16]:
Merged_layer.drop_duplicates(subset=['BlockNumber'], keep='first', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [17]:
print('  2.d Defining new Layer')

  2.d Defining new Layer


In [18]:
new_columns = np.dtype([
                        ("BlockUniqueID", int),
                        ('BlockNumber', int),
                        ("SubBlockNumber", int),
                        ("BlockStatus", int),
                        ("IsJordanian", int),
                        ("LandType", int),
                        #("SetteledDate", 'datetime64[D]'),
                        #("UpdateDate", 'datetime64[D]'),
                        #("LastRegisterdParcel", int),
                        #("LastParcel", int),
                        #("LastSetteledParcel", int),
                        #("LastCourtParcel", int),
                        ('County', str),
                        ('LocalityID', int),
                        ('Remarks', str)
                       ])

TaxAssassmentBlocks = pd.DataFrame(np.empty(0, dtype=new_columns))

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [19]:
TaxAssassmentBlocks['BlockUniqueID'] = Merged_layer['BlockUniqueID'].astype(int)


TaxAssassmentBlocks['BlockNumber'] = Merged_layer['BlockNumber'].astype(int)


TaxAssassmentBlocks['SubBlockNumber'] = Merged_layer['SubBlockNumber'].fillna(0)
TaxAssassmentBlocks['SubBlockNumber'] = TaxAssassmentBlocks['SubBlockNumber'].astype(int)


TaxAssassmentBlocks['BlockStatus'] = 1


TaxAssassmentBlocks['IsJordanian'] = 0


TaxAssassmentBlocks['LandType'] = 1


#TaxAssassmentBlocks['SetteledDate'] = None


#TaxAssassmentBlocks['UpdateDate'] = None


#TaxAssassmentBlocks['LastRegisterdParcel'] = None


#TaxAssassmentBlocks['LastParcel'] = None


#TaxAssassmentBlocks['LastSetteledParcel'] = None


#TaxAssassmentBlocks['LastCourtParcel'] = None


TaxAssassmentBlocks['County'] = Merged_layer['BOOK_COUNTY']


TaxAssassmentBlocks['LocalityID'] = Merged_layer['LOCALITY_ID'].astype('Int64')


TaxAssassmentBlocks['Remarks'] = Merged_layer['BOOK_COMMENT']


TaxAssassmentBlocks['geometry'] = Merged_layer['geometry']


TaxAssassmentBlocks.sort_values('BlockUniqueID', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  2.e Exporting results to product folder')

In [ ]:
TaxAssassmentBlocks_geom = TaxAssassmentBlocks[~TaxAssassmentBlocks['geometry'].isna()]
TaxAssassmentBlocks_no_geom = TaxAssassmentBlocks[TaxAssassmentBlocks['geometry'].isna()]

In [ ]:
no_geom_csv_dir = CNFG.CSVPath + 'TaxAssessmentBlocks_no_geom.csv'
json_dir        = CNFG.JsonPath + 'TaxAssessmentBlocks_geom.geojson'


TaxAssassmentBlocks_no_geom.drop(columns='geometry').to_csv(no_geom_csv_dir, index=False, encoding = 'cp1255')

gpd.GeoDataFrame(TaxAssassmentBlocks_geom, geometry = 'geometry',  crs = 2039).to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + "Tax Assessment Blocks-Save as Feature Class.bat"]);